In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sentence_transformers import SentenceTransformer

print("imported libraries")

/Users/Apple/Documents/Python-projects/Task Price Predictor/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imported libraries


In [71]:
# 📥 Load the datasets
service_applications = pd.read_csv("csv/final_v4_serviceApplications.csv")
service_task_proposals = pd.read_csv("csv/final_v4_serviceTaskProposals.csv")
service_requests = pd.read_csv("csv/final_v4_serviceRequests.csv")
service_tasks = pd.read_csv("csv/final_v4_serviceTasks.csv")

In [72]:
# 🎯 Step 1: Filter accepted applications
accepted_apps = service_applications[service_applications["status"] == "accepted"]
accepted_ids = accepted_apps["$id"].tolist()

# 🎯 Step 2: Filter valid proposals linked to accepted applications
valid_proposals = service_task_proposals[
    service_task_proposals["serviceApplication"].isin(accepted_ids)
]

# 🔗 Step 3: Merge with serviceTasks and serviceRequests
merged = valid_proposals.merge(
    service_tasks, left_on="serviceTask", right_on="$id", suffixes=("", "_task")
).merge(
    service_requests, left_on="serviceRequest", right_on="$id", suffixes=("", "_request")
)

In [73]:
# Étape 4 : Créer un DataFrame d’entraînement avec les bons champs
df = pd.DataFrame({
    "titre_prestation": merged["title_request"],
    "desc_prestation": merged["description_request"],
    "titre_tache": merged["title"],
    "desc_tache": merged["description"],
    "prix": merged["newPrice"]
})

# Remplacer les NaN éventuels
df = df.fillna("")

In [74]:
# 🧹 Step 4: Build the training DataFrame
df = pd.DataFrame({
    "titre_prestation": merged["title_request"],
    "desc_prestation": merged["description_request"],
    "titre_tache": merged["title"],
    "desc_tache": merged["description"],
    "prix": merged["newPrice"]
})

df = df.fillna("")  # Fill missing values

# 🧠 Step 5: Combine text fields into a single column
df["combined_text"] = df[["titre_prestation", "desc_prestation", "titre_tache", "desc_tache"]].agg(" ".join, axis=1)

# 🔢 Step 6: Use SentenceTransformer to encode text
model_embed = SentenceTransformer('all-MiniLM-L6-v2')
X_vectors = model_embed.encode(df["combined_text"].tolist())
y = df["prix"]

# 🎓 Step 7: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# 📈 Step 8: Train regression model
model = LinearRegression()
model.fit(X_train, y_train)

# 📊 Evaluate the model
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)

In [ ]:
# 🧪 Step 9: Example prediction
example = {
    "titre_prestation": "Install water heater",
    "desc_prestation": "Replace old heater with new electric model.",
    "titre_tache": "Urgent Fix leaking sink in the rooftop",
    "desc_tache": "Identify and repair kitchen sink leak. Location: rooftop, Quantity: 3, Type: urgent."
}
example["combined_text"] = " ".join([
    example["titre_prestation"],
    example["desc_prestation"],
    example["titre_tache"],
    example["desc_tache"]
])
example_vector = model_embed.encode([example["combined_text"]])
predicted_price = model.predict(example_vector)[0]

print(predicted_price)

np.float32(87.98071)